In [1]:
!pip install transformers

     ---------------------------------------- 6.8/6.8 MB 13.5 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 11.1 MB/s eta 0:00:00
     ------------------------------------- 199.8/199.8 kB 11.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")


D:\Anacondo\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\untaw\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [49]:
import pandas as pd
import csv

# train_dataset = pd.read_csv("QA_final.csv", index_col=0, header=None, squeeze=True).to_dict()
train_dataset = pd.read_csv("QA_dual.csv", index_col=0, header=None, squeeze=True).to_dict()

C:\Users\untaw\AppData\Local\Temp\ipykernel_29924\2513407375.py:5: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  train_dataset = pd.read_csv("QA_dual.csv", index_col=0, header=None, squeeze=True).to_dict()


In [58]:
# print(len("P-Z1-DB0-202 represents MULTINATIONAL NACALA ROAD CORRIDOR DEVELOPMENT PROJECT PHASE V - MALAWI happening in Multinational which has status as Identification ->"))
print(train_dataset)

{1: {'prompt': 'project code', 'P-Z1-DB0-202 represents MULTINATIONAL NACALA ROAD CORRIDOR DEVELOPMENT PROJECT PHASE V - MALAWI happening in Multinational which has status as Identification ->': 'P-Z1-DB0-202', 'P-LS-E00-007 represents CLIMATE CHANGE ADAPTATION FOR SUSTAINABLE RURAL WATER SUPPLY IN LOWLANDS LESOTHO happening in Lesotho which has status as Lending ->': 'P-LS-E00-007', "P-GA-E00-001 represents PROGRAMME INTÉGRÉ D'ALIMENTATION EN EAU POTABLE ET ASSAINISSEMENT DE LIBREVILLE SOUS-PROGRAMME 1 happening in Gabon which has status as Approved ->": 'P-GA-E00-001', 'P-UG-EB0-007 represents PREPARATION OF FEASIBILITY STUDIES AND DETAILED DESIGNS FOR FAECAL SLUDGE SERVICE CHAIN MANAGEMENT happening in Uganda which has status as Approved ->': 'P-UG-EB0-007', "P-DZ-KF0-003 represents APPUI GOUVERNANCE ENTREPRISES PUBLIQUES ET DES PARTICIPATIONS DE L'ETAT happening in Algeria which has status as Approved ->": 'P-DZ-KF0-003', 'P-CM-F00-006 represents NACHTIGAL HYDRO POWER PROJECT happe

In [59]:
from datasets import Dataset

clean_train_dataset = Dataset.from_dict(clean_train_dict)
eval_dataset = Dataset.from_dict(eval_dict)

print(clean_train_dataset)
print(eval_dataset)

Dataset({
    features: ['P-Z1-DB0-202 represents MULTINATIONAL NACALA ROAD CORRIDOR DEVELOPMENT PROJECT PHASE V - MALAWI happening in Multinational which has status as Identification ->', 'P-LS-E00-007 represents CLIMATE CHANGE ADAPTATION FOR SUSTAINABLE RURAL WATER SUPPLY IN LOWLANDS LESOTHO happening in Lesotho which has status as Lending ->', "P-GA-E00-001 represents PROGRAMME INTÉGRÉ D'ALIMENTATION EN EAU POTABLE ET ASSAINISSEMENT DE LIBREVILLE SOUS-PROGRAMME 1 happening in Gabon which has status as Approved ->", 'P-UG-EB0-007 represents PREPARATION OF FEASIBILITY STUDIES AND DETAILED DESIGNS FOR FAECAL SLUDGE SERVICE CHAIN MANAGEMENT happening in Uganda which has status as Approved ->', "P-DZ-KF0-003 represents APPUI GOUVERNANCE ENTREPRISES PUBLIQUES ET DES PARTICIPATIONS DE L'ETAT happening in Algeria which has status as Approved ->", 'P-CM-F00-006 represents NACHTIGAL HYDRO POWER PROJECT happening in Cameroon which has status as Completed ->', 'P-UG-FAB-007 represents ACHWA II 

In [61]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["prompt"], padding="max_length", truncation=True)

clean_train_dataset = clean_train_dataset.map(tokenize_function, batched=True, batch_size=len(clean_train_dataset))
eval_dataset = eval_dataset.map(tokenize_function, batched=True, batch_size=len(eval_dataset))

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

KeyError: 'prompt'

In [42]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=clean_train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()


D:\Anacondo\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,past_key_values,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,labels,use_cache,output_attentions,output_hidden_states,return_dict,label,label_ids,labels.